In [26]:
#Import Statements

import numpy as np


In [46]:
def forward(X,W,B):
    #y = wtranspose.x + biases
    print("Dimensions of X,W,B:",X.shape,W.shape,B.shape)
    Y = X.dot(W) + B
    
    #o = y
    O = sigmoid(Y)
    return O

In [50]:
def forward_pass(X,network):
    W = network.weights[0]
    B = network.biases[0]
    O = forward(X,W,B)
    for i in range(1,len(network.weights)):
        W = network.weights[i]
        B = network.biases[i]
        O = forward(O,W,B)
    return O

In [ ]:
def calc_cost(O,t):
    #Average of Error
    

In [ ]:
def backward_pass(O,t):
    pass

In [ ]:
#Activation Function
def ReLU(z):
    return np.maximum(z,0,z)

In [41]:
def sigmoid(z):
    return 1/(np.exp(-1*z) + 1)

In [27]:
class Network:
    def __init__(self,layers):
        if len(layers)<2:
            print("Cannot create Neural Network with less than 2 layers")
            return
        self.layers = layers
        self.weights = []
        self.biases = []
        
        for i in range(1, len(layers)):
            rows = layers[i-1] #784
            cols = layers[i] #28
            layer_weight = np.random.random((rows,cols))
            layer_bias = np.random.random(layers[i])
            self.weights.append(layer_weight)
            self.biases.append(layer_bias)

In [51]:
my_net = Network([3,1])
my_net.weights

[array([[ 0.04073864],
        [ 0.25581163],
        [ 0.33740176]])]

In [52]:
X = np.random.random((5,3))
O = forward_pass(X,my_net)

Dimensions of X,W,B: (5, 3) (3, 1) (1,)


In [55]:
O

array([[ 0.64860793],
       [ 0.55829617],
       [ 0.67742139],
       [ 0.63422029],
       [ 0.6349737 ]])

In [56]:
t = np.array([0.5,0.5,0.5,0.5,0.5])